In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_data =  "//content//drive//My Drive//BDMH//Assignment 3//trainset.data"
test_data  =  "//content//drive//My Drive//BDMH//Assignment 3//testset.dat"

In [0]:
def getdata(path):
  sequence = []
  label_convert = []

  with open(path) as f:
    for line in f:
      text_file = line.split(",")
      sequence.append(text_file[0])
      s = text_file[1]
      s = s[:-1]
      label_convert.append(s)

  sequence = sequence[1:]        
  label_convert = label_convert[1:] 

  label = []

  for item in label_convert:
    if item == '+1':
        label.append(1)
    else:
        label.append(-1)

  return sequence,label


In [0]:
X_tr, Y_train = getdata(train_data)

In [0]:
aminoacid_sequence = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']

dipeptide_seq = []

for charone in aminoacid_sequence:
  for chartwo in aminoacid_sequence:
    char_to_add = charone+chartwo
    dipeptide_seq.append(char_to_add)




In [0]:
# dipeptide_seq

In [0]:
def get_dipeptite_dictionary(str):
  size = len(str)
  i = 0

  eachseq_list = []
  while( i < size - 1 ):
    add_seq = str[i] + str[i+1]
    eachseq_list.append(add_seq)
    i += 1
  
  getdict = {}

  for item in eachseq_list:
    if getdict.get(item) == None:
      getdict[item] = 1
    else:
      temp = getdict[item]
      temp += 1
      getdict[item] = temp

  new_dict = {}
  for item in getdict:
    value = getdict[item]
    value = value/(len(str) - 1)
    new_dict[item] = value

  return new_dict

In [0]:
def getlist_of_dictionary(X_tr):
  list_dictionary = []

  for each in X_tr:
    dic = get_dipeptite_dictionary(each)
    list_dictionary.append(dic)

  return list_dictionary  

In [0]:
def getvector(vector_dictionary):
  final_vector = []

  for dic in vector_dictionary:

    vec = [0.0] * 400

    for item in dic:

      for seq in dipeptide_seq:
        if seq == item:
          index = dipeptide_seq.index(seq)
          vec[index] = dic[item]

    final_vector.append(vec)

  return final_vector


In [0]:
training_vector_dictionary = getlist_of_dictionary(X_tr)

In [0]:
X_train = getvector(training_vector_dictionary)

In [194]:
len(X_train)

2270

In [195]:
len(Y_train)

2270

In [0]:
x = np.array(X_train)
y = np.array(Y_train)

In [200]:
x.shape

(2270, 400)

In [201]:
y.shape

(2270,)

In [0]:
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size=0.3, random_state=42)

In [0]:
y_train_final = []
for i in y_train:
  if i == -1:
    y_train_final.append(0)
  else:
    y_train_final.append(1)

y_train_final = np.array(y_train_final)


In [0]:
y_val_final = []
for i in y_val:
  if i == -1:
    y_val_final.append(0)
  else:
    y_val_final.append(1)

y_val_final = np.array(y_val_final)


In [217]:
len(y_val_final)

681

In [218]:
len(y_train_final)

1589

In [0]:
## x_train
## x_val

## y_val_final
## y_train_final

In [0]:
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.optimizers import Adam

In [0]:
model = Sequential([
                    Dense(16, input_shape = (400,), activation='relu'),
                    Dense(32, activation = 'relu'),
                    Dense(2, activation = 'softmax')
                    ])

In [235]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 16)                6416      
_________________________________________________________________
dense_13 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 66        
Total params: 7,026
Trainable params: 7,026
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [237]:
model.fit(x_train,y_train_final,batch_size=10, epochs=15)

Epoch 1/15
159/159 [==============================] - 0s 1ms/step - loss: 0.6614 - accuracy: 0.6287
Epoch 2/15
159/159 [==============================] - 0s 1ms/step - loss: 0.6291 - accuracy: 0.6369
Epoch 3/15
159/159 [==============================] - 0s 1ms/step - loss: 0.5620 - accuracy: 0.6992
Epoch 4/15
159/159 [==============================] - 0s 1ms/step - loss: 0.4807 - accuracy: 0.7722
Epoch 5/15
159/159 [==============================] - 0s 1ms/step - loss: 0.4291 - accuracy: 0.8081
Epoch 6/15
159/159 [==============================] - 0s 1ms/step - loss: 0.3850 - accuracy: 0.8345
Epoch 7/15
159/159 [==============================] - 0s 1ms/step - loss: 0.3444 - accuracy: 0.8521
Epoch 8/15
159/159 [==============================] - 0s 1ms/step - loss: 0.3007 - accuracy: 0.8754
Epoch 9/15
159/159 [==============================] - 0s 1ms/step - loss: 0.2605 - accuracy: 0.8968
Epoch 10/15
159/159 [==============================] - 0s 1ms/step - loss: 0.2196 - accuracy: 0.9226

In [238]:
y_val_predict = model.predict_classes(x_val)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [239]:
y_val_predict

array([1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,

In [240]:
y_val_final

array([0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,

In [0]:
from sklearn.metrics import accuracy_score

In [243]:
accuracy_score(y_val_final,y_val_predict)

0.7151248164464024

In [0]:
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.layers import Flatten

In [0]:
model_new = Sequential([Flatten(),
                    Dense(512, activation='relu'),
                    Dense(256, activation = 'relu'),
                    Dense(128, activation = 'relu'),
                    Dense(2, activation = 'softmax')
                    ])

In [0]:
model_new.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [298]:
model_new.fit(x_train,y_train_final, epochs=15)

Epoch 1/15
50/50 [==============================] - 1s 17ms/step - loss: 0.6255 - accuracy: 0.6375
Epoch 2/15
50/50 [==============================] - 1s 20ms/step - loss: 0.4549 - accuracy: 0.7904
Epoch 3/15
50/50 [==============================] - 1s 19ms/step - loss: 0.2644 - accuracy: 0.8962
Epoch 4/15
50/50 [==============================] - 1s 20ms/step - loss: 0.1187 - accuracy: 0.9629
Epoch 5/15
50/50 [==============================] - 1s 21ms/step - loss: 0.0366 - accuracy: 0.9899
Epoch 6/15
50/50 [==============================] - 1s 19ms/step - loss: 0.0174 - accuracy: 0.9975
Epoch 7/15
50/50 [==============================] - 1s 18ms/step - loss: 0.0031 - accuracy: 0.9994
Epoch 8/15
50/50 [==============================] - 1s 17ms/step - loss: 0.0011 - accuracy: 1.0000
Epoch 9/15
50/50 [==============================] - 1s 19ms/step - loss: 6.7582e-04 - accuracy: 1.0000
Epoch 10/15
50/50 [==============================] - 1s 17ms/step - loss: 4.8513e-04 - accuracy: 1.0000
E

In [0]:
y_val_predict = model_new.predict_classes(x_val)

In [300]:
accuracy_score(y_val_final,y_val_predict)

0.7665198237885462

In [0]:
## Now Test with test data

In [0]:
test_data  =  "//content//drive//My Drive//BDMH//Assignment 3//testset.dat"

In [0]:
test_seq = []
with open(test_data) as f:
  for line in f:
    text_file = line.split(",")
    test_seq.append(text_file[1][:-1])

X_te = test_seq[1:]

In [0]:
# X_te

In [0]:
testing_vector_dictionary = getlist_of_dictionary(X_te)

In [0]:
X_test = getvector(testing_vector_dictionary)

In [0]:
X_test = np.array(X_test)

In [0]:
Y_predict =  model_new.predict_classes(X_test)

In [0]:
final_test = []
for i in Y_predict:
  if i == 0:
    final_test.append(-1)
  else:
    final_test.append(1)  


In [319]:
count = 1001
for i in final_test:
  print(count,',',i, sep='')
  count += 1

1001,-1
1002,1
1003,-1
1004,-1
1005,-1
1006,-1
1007,-1
1008,1
1009,-1
1010,1
1011,-1
1012,-1
1013,1
1014,-1
1015,-1
1016,-1
1017,-1
1018,-1
1019,1
1020,-1
1021,-1
1022,1
1023,-1
1024,-1
1025,-1
1026,-1
1027,-1
1028,-1
1029,-1
1030,-1
1031,1
1032,-1
1033,1
1034,-1
1035,1
1036,-1
1037,-1
1038,-1
1039,1
1040,1
1041,-1
1042,-1
1043,1
1044,1
1045,-1
1046,-1
1047,1
1048,-1
1049,-1
1050,1
1051,-1
1052,-1
1053,-1
1054,-1
1055,-1
1056,-1
1057,-1
1058,-1
1059,1
1060,1
1061,-1
1062,1
1063,-1
1064,1
1065,-1
1066,-1
1067,-1
1068,1
1069,-1
1070,-1
1071,1
1072,1
1073,-1
1074,1
1075,-1
1076,-1
1077,-1
1078,-1
1079,-1
1080,1
1081,-1
1082,1
1083,-1
1084,-1
1085,1
1086,-1
1087,-1
1088,1
1089,-1
1090,-1
1091,1
1092,-1
1093,1
1094,-1
1095,1
1096,1
1097,-1
1098,1
1099,-1
1100,1
1101,-1
1102,-1
1103,1
1104,-1
1105,-1
1106,-1
1107,-1
1108,-1
1109,1
1110,-1
1111,-1
1112,-1
1113,-1
1114,-1
1115,-1
1116,1
1117,1
1118,1
1119,-1
1120,-1
1121,-1
1122,-1
1123,-1
1124,-1
1125,-1
1126,-1
1127,-1
1128,-1
1129,1
1130,-1